In [16]:
import os
import numpy as np
from numpy.random import seed
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Conv2D, BatchNormalization, Input, Average, MaxPool2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.activations import relu, softmax, tanh
from tensorflow.keras.initializers import he_normal, glorot_uniform
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.regularizers import l2, l1, l1_l2
from tensorflow.random import set_seed
from tensorflow import keras

In [17]:
print("Version de TensorFlow :", tf.__version__)
print("Nom du GPU :", tf.test.gpu_device_name())

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(False)

Version de TensorFlow : 2.7.0
Nom du GPU : /device:GPU:0


2022-01-06 18:20:17.287857: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-06 18:20:17.287911: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [18]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [19]:
NUM_CLASSES = 10
IMG_SIZE = x_train[0].shape

In [20]:
x_train = x_train.astype('float32') / 256
x_test = x_test.astype('float32') / 256

y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

In [21]:
LOG_DIR = os.path.join("logs")

In [22]:
set_seed(42) # TensorFlow
seed(42) # NumPy

In [25]:
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True
LEARNING_RATE = 0.01
MOMENTUM = 0.1
FILTERS = 32

In [28]:
def Resnet():
    input_tensor = keras.layers.Input(shape=(32, 32, 3))
    hidden_tensor = input_tensor

    for i in range(10):
        prev_tensor = hidden_tensor
        hidden_tensor = keras.layers.Conv2D(32, (3, 3), padding='same', activation=keras.activations.relu,
                                            kernel_initializer=keras.initializers.he_normal)(hidden_tensor)
        #hidden_tensor = keras.layers.Dropout(0.2)(hidden_tensor)
        if i > 0:
            hidden_tensor = keras.layers.Add()([hidden_tensor, prev_tensor]) / 2.0
        else:
            hidden_tensor = keras.layers.Add()([hidden_tensor, keras.layers.Dense(32)(prev_tensor)]) / 2.0

    hidden_tensor = keras.layers.Flatten()(hidden_tensor)
    output_tensor = keras.layers.Dense(10, activation=keras.activations.softmax,
                                       kernel_regularizer=keras.regularizers.l2(0.01),
                                       bias_regularizer=keras.regularizers.l2(0.01))(hidden_tensor)

    model = keras.models.Model(input_tensor, output_tensor)
    return model

In [29]:
resnet = Resnet() 
resnet.compile(loss=categorical_crossentropy,
                  optimizer=SGD(LEARNING_RATE,MOMENTUM),
                  metrics= categorical_accuracy)
RESNET_LOG = os.path.join(LOG_DIR, "resnet",
                        f"test_modele_prof")
resnet.fit(x_train,y_train,
              batch_size=BATCH_SIZE,
              epochs=EPOCHS,
              validation_data=(x_test, y_test),
              shuffle=SHUFFLE,
              callbacks=[tf.keras.callbacks.TensorBoard(RESNET_LOG, histogram_freq=1)])

Epoch 1/200


2022-01-06 18:21:35.626669: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


98/98 [==============================] - ETA: 0s - loss: 2.4911 - categorical_accuracy: 0.1191

2022-01-06 18:22:16.732157: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


98/98 [==============================] - 45s 422ms/step - loss: 2.4911 - categorical_accuracy: 0.1191 - val_loss: 2.4762 - val_categorical_accuracy: 0.1261
Epoch 2/200
98/98 [==============================] - 42s 424ms/step - loss: 2.4348 - categorical_accuracy: 0.1686 - val_loss: 2.3608 - val_categorical_accuracy: 0.1976
Epoch 3/200
98/98 [==============================] - 41s 419ms/step - loss: 2.3207 - categorical_accuracy: 0.2177 - val_loss: 2.2940 - val_categorical_accuracy: 0.2524
Epoch 4/200
98/98 [==============================] - 41s 419ms/step - loss: 2.2768 - categorical_accuracy: 0.2408 - val_loss: 2.2688 - val_categorical_accuracy: 0.2621
Epoch 5/200
98/98 [==============================] - 42s 434ms/step - loss: 2.2422 - categorical_accuracy: 0.2629 - val_loss: 2.2237 - val_categorical_accuracy: 0.2724
Epoch 6/200
98/98 [==============================] - 45s 458ms/step - loss: 2.2072 - categorical_accuracy: 0.2779 - val_loss: 2.1883 - val_categorical_accuracy: 0.2915
Epoc

98/98 [==============================] - 61s 626ms/step - loss: 1.4145 - categorical_accuracy: 0.5171 - val_loss: 1.4220 - val_categorical_accuracy: 0.5107
Epoch 50/200
98/98 [==============================] - 70s 714ms/step - loss: 1.4080 - categorical_accuracy: 0.5194 - val_loss: 1.4497 - val_categorical_accuracy: 0.5074
Epoch 51/200
98/98 [==============================] - 58s 587ms/step - loss: 1.3938 - categorical_accuracy: 0.5232 - val_loss: 1.4528 - val_categorical_accuracy: 0.5015
Epoch 52/200
98/98 [==============================] - 57s 585ms/step - loss: 1.3927 - categorical_accuracy: 0.5251 - val_loss: 1.4412 - val_categorical_accuracy: 0.5033
Epoch 53/200
98/98 [==============================] - 57s 585ms/step - loss: 1.3786 - categorical_accuracy: 0.5300 - val_loss: 1.4926 - val_categorical_accuracy: 0.4890
Epoch 54/200
98/98 [==============================] - 58s 589ms/step - loss: 1.3786 - categorical_accuracy: 0.5281 - val_loss: 1.3948 - val_categorical_accuracy: 0.5185

98/98 [==============================] - 60s 610ms/step - loss: 1.1374 - categorical_accuracy: 0.6188 - val_loss: 1.2082 - val_categorical_accuracy: 0.5926
Epoch 98/200
98/98 [==============================] - 59s 602ms/step - loss: 1.1374 - categorical_accuracy: 0.6204 - val_loss: 1.1925 - val_categorical_accuracy: 0.5962
Epoch 99/200
98/98 [==============================] - 63s 638ms/step - loss: 1.1360 - categorical_accuracy: 0.6216 - val_loss: 1.2347 - val_categorical_accuracy: 0.5745
Epoch 100/200
98/98 [==============================] - 66s 676ms/step - loss: 1.1310 - categorical_accuracy: 0.6214 - val_loss: 1.2033 - val_categorical_accuracy: 0.5910
Epoch 101/200
98/98 [==============================] - 62s 630ms/step - loss: 1.1195 - categorical_accuracy: 0.6281 - val_loss: 1.2860 - val_categorical_accuracy: 0.5716
Epoch 102/200
98/98 [==============================] - ETA: 0s - loss: 1.1200 - categorical_accuracy: 0.6277

KeyboardInterrupt: 